In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv('data\diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

In [5]:
#here few misconception is there lke BMI can not be zero, BP can't be zero, glucose, insuline can't be zero so lets try to fix it
# now replacing zero values with the mean of the column
df['BMI'] = df['BMI'].replace(0,df['BMI'].mean())
df['BloodPressure'] = df['BloodPressure'].replace(0,df['BloodPressure'].mean())
df['Glucose'] = df['Glucose'].replace(0,df['Glucose'].mean())
df['Insulin'] = df['Insulin'].replace(0,df['Insulin'].mean())
df['SkinThickness'] = df['SkinThickness'].replace(0,df['SkinThickness'].mean())

In [6]:
X=df.drop(columns=['Outcome'])
y=df['Outcome']

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=30)

In [28]:
import os
import pickle
from sklearn.preprocessing import StandardScaler
def scaler_standard(X_train,X_test):
    scaler=StandardScaler()
    X_train_scaled=scaler.fit_transform(X_train)
    X_test_scaled=scaler.transform(X_test)
    
    directory_path = os.path.join('..', 'Model')
    os.makedirs(directory_path, exist_ok=True)
    file_path = os.path.join(directory_path, 'standardScaler.pkl')
    with open(file_path, 'wb') as file:
        pickle.dump(scaler, file)
    return X_train_scaled,X_test_scaled

In [29]:
X_train_scaled,X_test_scaled=scaler_standard(X_train,X_test)

In [30]:
X_test_scaled

array([[-1.14725293, -0.69157347,  0.29807283, ...,  0.44634363,
        -0.82771153, -0.62840783],
       [-0.24326156, -0.52653899, -1.55162953, ..., -0.23350617,
        -0.55306398, -0.79505742],
       [ 1.86605161, -1.54975281,  0.80253711, ...,  0.10641873,
        -0.63779567,  0.37148971],
       ...,
       [-0.54459202,  0.10059207, -1.04716525, ...,  0.17723642,
        -1.14910759, -0.21178385],
       [ 0.05806889,  0.49667484,  0.97069187, ..., -0.19101555,
        -0.66993528, -0.29510865],
       [ 0.6607298 ,  0.85975071, -0.03823669, ...,  0.14890934,
         0.42573311,  1.37138725]])

In [32]:
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(X_train_scaled,y_train)

LogisticRegression()

In [33]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
parameters={
    'penalty':['l1','l2'],
    'C':np.logspace(-3,3,7),
    'solver':['newton-cg','lbfgs','liblinear'],
}

In [34]:
logreg=LogisticRegression()
clf=GridSearchCV(logreg,param_grid=parameters,
                 scoring='accuracy',
                 cv=10)
clf.fit(X_train_scaled,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring='accuracy')

In [35]:
clf.best_params_

{'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}

In [36]:
clf.best_score_

0.768995765275257

In [37]:
y_pred=clf.predict(X_test_scaled)

In [39]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
conf_mat=confusion_matrix(y_test,y_pred)
conf_mat

array([[115,  15],
       [ 27,  35]], dtype=int64)

In [41]:
directory_path = os.path.join('..', 'Model')
os.makedirs(directory_path, exist_ok=True)
file_path = os.path.join(directory_path, 'modelForPrediction.pkl')
with open(file_path, 'wb') as file:
    pickle.dump(log_reg, file)